In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LinearRegression

from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import VotingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report


import glob
import os

# for reproduceability in model evaluation
rand_state = 42
np.random.seed(rand_state)

#merge all files starting in "part-0002" & ending in "csv" inside the selected path
all_files = os.path.join("CIC-IoT-2023/","part-0001*.csv")
#list of all merged files
joined_files = glob.glob(all_files)
#merged files
df = pd.concat(map(pd.read_csv, joined_files), ignore_index=True)
# pd.read_csv(joined_files, chunksize=1000)
#df = pd.read_csv(df, chunksize=1000)
# for chunk in pd.read_csv(joined_files, chunksize=1000):
#     print(chunk.shape)

df.info()
df_test = df.head(5)
df_test

# df = pd.read_csv('part-00000-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv')
# df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356866 entries, 0 to 2356865
Data columns (total 47 columns):
 #   Column           Dtype  
---  ------           -----  
 0   flow_duration    float64
 1   Header_Length    float64
 2   Protocol Type    float64
 3   Duration         float64
 4   Rate             float64
 5   Srate            float64
 6   Drate            float64
 7   fin_flag_number  float64
 8   syn_flag_number  float64
 9   rst_flag_number  float64
 10  psh_flag_number  float64
 11  ack_flag_number  float64
 12  ece_flag_number  float64
 13  cwr_flag_number  float64
 14  ack_count        float64
 15  syn_count        float64
 16  fin_count        float64
 17  urg_count        float64
 18  rst_count        float64
 19  HTTP             float64
 20  HTTPS            float64
 21  DNS              float64
 22  Telnet           float64
 23  SMTP             float64
 24  SSH              float64
 25  IRC              float64
 26  TCP              float64
 27  UDP         

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,54.0,6.00,64.00,27.739647,27.739647,0.0,0.0,0.0,0.0,...,0.000000,54.0,8.333125e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DDoS-PSHACK_Flood
1,0.005011,6831.0,16.83,63.36,27292.469571,27292.469571,0.0,0.0,0.0,0.0,...,0.377989,50.1,8.301666e+07,9.5,10.011452,0.535314,1.045327,0.14,141.55,DoS-UDP_Flood
2,4.539816,108.0,6.00,64.00,0.440547,0.440547,0.0,0.0,0.0,0.0,...,0.000000,54.0,8.295607e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DoS-TCP_Flood
3,4.501544,108.0,6.00,64.00,0.444292,0.444292,0.0,0.0,0.0,0.0,...,0.000000,54.0,8.294753e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,DoS-TCP_Flood
4,0.186995,268413.0,17.00,64.00,2597.167113,2597.167113,0.0,0.0,0.0,0.0,...,0.000000,554.0,8.378470e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,Mirai-udpplain


In [2]:
df['label'].describe()

count             2356866
unique                 34
top       DDoS-ICMP_Flood
freq               364373
Name: label, dtype: object

In [3]:
# samples, features+index
df.shape

(2356866, 47)

In [4]:
# count null values
null_vals = df.isnull().sum()
if null_vals.sum()>0:
    print("Null values detected!")
else:
    print("No null values detected.")

No null values detected.


In [5]:
# count duplicate values
dupe_vals = df.duplicated().sum()
if dupe_vals.sum() > 0:
    print("Duplicate values detected!")
    print("Removing duplicates...")
    df = df.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicate values detected.")


No duplicate values detected.


In [6]:
# check data types
#print(df.dtypes)

In [7]:
# PREPARE DATA
# drop duplicates
#df = df.drop_duplicates()

#create input data
X = df.drop(columns=['label'])

#create output (prediction) data
y = df['label']

In [8]:
#split data - 80% training, 20% testing
# training input data, testing input data, training output data, testing output data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

In [9]:
# Baseline Random Classifier
dummy_classifier = DummyClassifier(strategy='stratified', random_state=rand_state)
# Fit baseline classifier on training data
dummy_classifier.fit(X_train, y_train)
# Predictions from test data
y_pred_dummy = dummy_classifier.predict(X_test)

dummy_acc = accuracy_score(y_test, y_pred_dummy)
report = classification_report(y_test, y_pred_dummy)

print("Baseline Classifier Accuracy: ", dummy_acc)
print("Classification Report:", report)

# save output to file
with open("baselineClassificationReport.txt", "a") as file:
    print("Baseline Classifier Accuracy: ", dummy_acc, "\nClassification Report:", report, file=file)

Baseline Classifier Accuracy:  0.08740193561800184
Classification Report:                          precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.00      0.00        36
          BenignTraffic       0.02      0.02      0.02     11126
       BrowserHijacking       0.00      0.00      0.00        63
       CommandInjection       0.00      0.00      0.00        51
 DDoS-ACK_Fragmentation       0.01      0.01      0.01      2879
        DDoS-HTTP_Flood       0.00      0.00      0.00       296
        DDoS-ICMP_Flood       0.15      0.15      0.15     72580
DDoS-ICMP_Fragmentation       0.01      0.01      0.01      4634
      DDoS-PSHACK_Flood       0.09      0.09      0.09     41168
       DDoS-RSTFINFlood       0.09      0.09      0.09     40692
         DDoS-SYN_Flood       0.09      0.09      0.09     41240
         DDoS-SlowLoris       0.00      0.00      0.00       221
DDoS-SynonymousIP_Flood       0.08      0.08      0.08     36155
         DDoS-T

In [10]:
# RESHAPE DATA from 3d to 2d
#nx, ny = X_train.shape
#d2_train_dataset = X_train.values.reshape(nx,ny)
#d2_train_dataset
models = {}
results = {}

In [11]:
# models = [
#     ('dt', DecisionTreeClassifier()),
#     ('rf', RandomForestClassifier())
# ]

In [18]:
# Robustness Check - Cross Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)

def evaluate_model(name, model, X_train, y_train, X_test, y_test):
    # Train model
    model.fit(X_train, y_train)
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    # Scoring
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    roc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv).mean()

    # with open(f"{model}Scoring.txt", "a") as file:
    #     print(f"\n{name} Results: \nAccuracy: {acc:.4f}\nF1: {f1:.4f}\nROC-AUC: {roc:.4f}\nCV: {cv_scores:.4f}\n\nClassification Report: \n", file=file)
    #     print(classification_report(y_test, y_pred), file=file)

    
    print(f"\n{name} Results: ")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1: {f1:.4f}")
    print(f"ROC-AUC: {roc:.4f}")
    print(f"CV: {cv_scores:.4f}")
    print(f"\nClassification Report: ")
    print(classification_report(y_test, y_pred))

    return{
        'model': model,
        'accuracy': acc,
        'f1_score': f1,
        'roc_auc': roc,
        'cv': cv_scores,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

In [19]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=rand_state,
    n_jobs=-1
)
results['Random Forest'] = evaluate_model('Random Forest', rf_model, X_train, y_train, X_test, y_test)


Random Forest Results: 
Accuracy: 0.9816
F1: 0.9795
ROC-AUC: 0.9728
CV: 0.9720

Classification Report: 


C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


                         precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.00      0.00        36
          BenignTraffic       0.64      1.00      0.78     11126
       BrowserHijacking       0.00      0.00      0.00        63
       CommandInjection       0.00      0.00      0.00        51
 DDoS-ACK_Fragmentation       1.00      0.98      0.99      2879
        DDoS-HTTP_Flood       0.97      0.61      0.75       296
        DDoS-ICMP_Flood       1.00      1.00      1.00     72580
DDoS-ICMP_Fragmentation       1.00      0.98      0.99      4634
      DDoS-PSHACK_Flood       1.00      1.00      1.00     41168
       DDoS-RSTFINFlood       1.00      1.00      1.00     40692
         DDoS-SYN_Flood       0.98      0.99      0.98     41240
         DDoS-SlowLoris       1.00      0.08      0.15       221
DDoS-SynonymousIP_Flood       1.00      0.99      0.99     36155
         DDoS-TCP_Flood       1.00      1.00      1.00     45472
         DDoS-UDP_Flood 

C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [20]:
dt_model = tree.DecisionTreeClassifier(
    max_depth=15,
    random_state=rand_state
)
results['Decision Tree'] = evaluate_model('Decision Tree', dt_model, X_train, y_train, X_test, y_test)


DecisionTree Results: 
Accuracy: 0.9894
F1: 0.9893
ROC-AUC: 0.9610
CV: 0.9895

Classification Report: 


C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


                         precision    recall  f1-score   support

       Backdoor_Malware       0.50      0.03      0.05        36
          BenignTraffic       0.87      0.98      0.92     11126
       BrowserHijacking       0.67      0.06      0.12        63
       CommandInjection       1.00      0.14      0.24        51
 DDoS-ACK_Fragmentation       0.99      0.93      0.96      2879
        DDoS-HTTP_Flood       1.00      0.17      0.29       296
        DDoS-ICMP_Flood       1.00      1.00      1.00     72580
DDoS-ICMP_Fragmentation       1.00      0.97      0.99      4634
      DDoS-PSHACK_Flood       1.00      1.00      1.00     41168
       DDoS-RSTFINFlood       1.00      1.00      1.00     40692
         DDoS-SYN_Flood       1.00      1.00      1.00     41240
         DDoS-SlowLoris       0.28      0.24      0.26       221
DDoS-SynonymousIP_Flood       1.00      1.00      1.00     36155
         DDoS-TCP_Flood       1.00      1.00      1.00     45472
         DDoS-UDP_Flood 

C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [21]:
# from sklearn.metrics import roc_curve, auc

# # Calculate ROC curve
# fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba) 
# roc_auc = auc(fpr, tpr)
# # Plot the ROC curve
# plt.figure()  
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve for Breast Cancer Classification')
# plt.legend()
# plt.show()

In [22]:
# dt = tree.DecisionTreeClassifier()
# rf = RandomForestClassifier()

# dt.fit(X_train, y_train, sample_weight=None)
# rf.fit(X_train, y_train, sample_weight=None)

In [23]:
# #enter data you want a prediction for
# rf_predictions = rf.predict(X_test)
# dt_predictions = dt.predict(X_test)
# rf_predictions

In [26]:
print("Model Comparison")
print("-"*40)

comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[m]['accuracy'] for m in results],
    'F1': [results[m]['f1_score'] for m in results],
    'ROC-AUC': [results[m]['roc_auc'] for m in results],
    'CV': [results[m]['cv'] for m in results]
})

#comparison = comparison.sort_values('F1', ascending=False)
print('\n', comparison.to_string(index=False))

# Best model according to F1 score
best_model_name = comparison.iloc[0]['Model']
best_model_result = results[best_model_name]
best_model = best_model_result['model']

print(f"\nBest Model: {best_model_name}")
print(f"Accuracy: {best_model_result['accuracy']:.4f}")
print(f"F1: {best_model_result['f1_score']:.4f}")
print(f"ROC-AUC: {best_model_result['roc_auc']:.4f}")
print(f"CV: {best_model_result['cv']:.4f}")

Model Comparison
----------------------------------------

         Model  Accuracy       F1  ROC-AUC       CV
Random Forest  0.981643 0.979544 0.972780 0.972018
 DecisionTree  0.989429 0.989264 0.960992 0.989476

Best Model: Random Forest
Accuracy: 0.9816
F1: 0.9795
ROC-AUC: 0.9728
CV: 0.9720


In [27]:
# VOTING CLASSIFIER
vote_hard = VotingClassifier(estimators=[('dt',dt_model),('rf',rf_model)], voting='hard')
vote_hard.fit(X_train, y_train)

# PREDICT
y_pred_vote = vote_hard.predict(X_test)


# dt = tree.DecisionTreeClassifier()
# rf = RandomForestClassifier()

# dt.fit(X_train, y_train, sample_weight=None)
# rf.fit(X_train, y_train, sample_weight=None)

# Make dt PERSISTANT
#store TRAINED dt in file called IDS_IoT.joblib
#joblib.dump(dt_model, 'IDS_IoT.joblib')
#load trained dt from file
#dt = joblib.load('IDS_IoT.joblib')

#enter data you want a prediction for
rf_predictions = rf_model.predict(X_test)
dt_predictions = dt_model.predict(X_test)
# rf_predictions

In [29]:
# SCORING
print("-"*60)
print("HARD VOTING")
print("-"*60)

accur_score = accuracy_score(y_test, y_pred_vote)
print("Accuracy: ", accur_score)

prec_score = precision_score(y_test, y_pred_vote, average='weighted') # "ill-defined" Warning
print("Precision Score: ", prec_score)

recall = recall_score(y_test, y_pred_vote, average='weighted')
print("Recall Score: ", recall)

f1 = f1_score(y_test, y_pred_vote, average='weighted')
print("F1 Score: ", f1)

------------------------------------------------------------
HARD VOTING
------------------------------------------------------------
Accuracy:  0.983130168401312


C:\Users\Ethan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Precision Score:  0.9843485352024453
Recall Score:  0.983130168401312
F1 Score:  0.9811856528349926


In [43]:
# #SCORING
# rf_accur_score = accuracy_score(y_test, rf_predictions) #returns # from 0-1
# dt_accur_score = accuracy_score(y_test, dt_predictions)

# rf_precision = precision_score(y_test, rf_predictions, average='weighted')
# dt_precision = precision_score(y_test, dt_predictions, average='weighted')

# rf_recall = recall_score(y_test, rf_predictions, average='weighted')
# dt_recall = recall_score(y_test, dt_predictions, average='weighted')

# print("Random Forest\nAccuracy: ", rf_accur_score)
# print("Precision: ", rf_precision)
# print("Recall: ", rf_recall)
# print("-----------------------------------------")
# print("Decision Tree\nAccuracy: ", dt_accur_score)
# print("Precision: ", dt_precision)
# print("Recall: ", dt_recall)


In [30]:
# VISUALIZATIONS
#decisionTree, filename, display specified columns & rules, output data in alphabetical order, writes info on graph, round box edges, color boxes
tree.export_graphviz(dt_model, out_file='IDS_IoT_decisionTree.dot', feature_names=X_test.columns, class_names=sorted(y.unique()), max_depth=5, label='all', rounded=True, filled=True)


In [31]:
rf_individual_tree = rf_model.estimators_[0] # grabs the first tree in the RandomForest
tree.export_graphviz(rf_individual_tree, out_file='IDS_IoT_rfTree.dot', feature_names=X_test.columns, class_names=sorted(y.unique()), max_depth=5, label='all', rounded=True, filled=True)
